# 02. Data Cleaning and Preprocessing

**Author**: Rafsamjani Anugrah  
**Date**: 2024  
**Project**: Credit Risk Prediction - ID/X Partners  

## Tujuan Notebook

Notebook ini berfokus pada membersihkan dan preprocessing data:
1. Filter dataset untuk loan yang selesai (Fully Paid/Charged Off)
2. Handle missing values dengan strategi yang tepat
3. Konversi tipe data (percentages, dates, currencies)
4. Create derived features (FICO average, financial ratios)
5. Handle outliers
6. Encoding categorical variables

## Prerequisites

- Dataset sudah dieksplorasi di notebook 01
- Environment sudah di-setup dengan libraries yang dibutuhkan

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import os
import json
from tqdm import tqdm

# Set styling
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Display options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

print("🔧 Data Cleaning Tools Loaded!")
print(f"📅 Cleaning started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. Load Dataset and Initial Setup

In [ ]:
# Load dataset
data_paths = [
    '../data/raw/loan_data_2007_2014.csv',
    '../../data/raw/loan_data_2007_2014.csv',
    'data/raw/loan_data_2007_2014.csv',
    'loan_data_2007_2014.csv'
]

data_path = None
for path in data_paths:
    if os.path.exists(path):
        data_path = path
        break

if data_path:
    print(f"✅ Loading dataset from: {data_path}")
    
    # Load dataset
    df = pd.read_csv(data_path, low_memory=False)
    
    print(f"📊 Dataset loaded successfully!")
    print(f"   Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    
    # Create backup of original data
    df_original = df.copy()
    print(f"💾 Original data backup created")
    
else:
    print("❌ Dataset not found!")
    print("Please ensure loan_data_2007_2014.csv is in the data/raw/ directory")
    df = None

## 2. Filter for Completed Loans

In [ ]:
if df is not None and 'loan_status' in df.columns:
    print("="*80)
    print("STEP 1: FILTER COMPLETED LOANS")
    print("="*80)
    
    # Analisis loan status sebelum filtering
    print("\n📊 Original Loan Status Distribution:")
    status_counts = df['loan_status'].value_counts()
    for status, count in status_counts.items():
        percentage = count / len(df) * 100
        print(f"   {status:<25}: {count:>7,} ({percentage:>6.2f}%)")
    
    # Define completed loans
    completed_loans = ['Fully Paid', 'Charged Off']
    
    print(f"\n🎯 Filtering for completed loans: {', '.join(completed_loans)}")
    
    # Filter dataset
    df_filtered = df[df['loan_status'].isin(completed_loans)].copy()
    
    print(f"\n📈 Filtering Results:")
    print(f"   Original dataset: {len(df):,} records")
    print(f"   Completed loans:   {len(df_filtered):,} records")
    print(f"   Records removed:   {len(df) - len(df_filtered):,} ({(len(df) - len(df_filtered))/len(df)*100:.1f}%)")
    
    # Analysis of completed loans
    completed_status_counts = df_filtered['loan_status'].value_counts()
    print(f"\n📊 Completed Loans Distribution:")
    for status, count in completed_status_counts.items():
        percentage = count / len(df_filtered) * 100
        print(f"   {status:<25}: {count:>7,} ({percentage:>6.2f}%)")
    
    # Calculate default rate
    default_rate = (df_filtered['loan_status'] == 'Charged Off').mean() * 100
    print(f"\n📈 Default Rate in Completed Loans: {default_rate:.2f}%")
    print(f"📈 Success Rate in Completed Loans: {100-default_rate:.2f}%")
    
    # Create binary target variable
    df_filtered['loan_status_binary'] = (df_filtered['loan_status'] == 'Charged Off').astype(int)
    
    print(f"\n✅ Target variable created:")
    print(f"   0 = Fully Paid (Good loans): {(df_filtered['loan_status_binary'] == 0).sum():,}")
    print(f"   1 = Charged Off (Bad loans): {(df_filtered['loan_status_binary'] == 1).sum():,}")
    
    # Check class imbalance
    imbalance_ratio = (df_filtered['loan_status_binary'] == 0).sum() / (df_filtered['loan_status_binary'] == 1).sum()
    print(f"\n⚖️  Class Imbalance Ratio: {imbalance_ratio:.1f}:1 (Non-default:Default)")
    
    # Update main dataframe
    df = df_filtered
    print(f"\n🔄 Main dataframe updated with filtered data")
    
else:
    print("❌ Dataset not available or loan_status column missing")

## 3. Data Type Conversion

In [ ]:
if df is not None:
    print("="*80)
    print("STEP 2: DATA TYPE CONVERSION")
    print("="*80)
    
    # Create a copy for cleaning
    df_clean = df.copy()
    conversions_made = 0
    
    print("\n🔄 Converting data types...")
    
    # 1. Convert percentage columns
    percentage_cols = ['int_rate', 'revol_util']
    print(f"\n📊 Converting percentage columns:")
    
    for col in percentage_cols:
        if col in df_clean.columns:
            print(f"   Processing {col}...")
            
            # Check original data
            sample_original = df_clean[col].dropna().head(3).astype(str).tolist()
            print(f"     Original sample: {sample_original}")
            
            # Convert
            df_clean[col] = df_clean[col].astype(str).str.replace('%', '')
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
            
            # Check converted data
            sample_converted = df_clean[col].dropna().head(3).tolist()
            print(f"     Converted sample: {sample_converted}")
            print(f"     Data type: {df_clean[col].dtype}")
            
            conversions_made += 1
        else:
            print(f"   ⚠️  Column {col} not found")
    
    # 2. Convert date columns
    date_cols = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']
    print(f"\n📅 Converting date columns:")
    
    for col in date_cols:
        if col in df_clean.columns:
            print(f"   Processing {col}...")
            
            # Check original data
            sample_original = df_clean[col].dropna().head(3).astype(str).tolist()
            print(f"     Original sample: {sample_original}")
            
            # Convert using pandas datetime with format specification
            df_clean[col] = pd.to_datetime(df_clean[col], format='%b-%y', errors='coerce')
            
            # Check converted data
            if df_clean[col].notna().any():
                sample_converted = df_clean[col].dropna().head(3).dt.strftime('%Y-%m-%d').tolist()
                print(f"     Converted sample: {sample_converted}")
                print(f"     Data type: {df_clean[col].dtype}")
                print(f"     Date range: {df_clean[col].min()} to {df_clean[col].max()}")
                conversions_made += 1
            else:
                print(f"     ⚠️  No valid dates found in {col}")
        else:
            print(f"   ⚠️  Column {col} not found")
    
    # 3. Convert employment length to numeric
    if 'emp_length' in df_clean.columns:
        print(f"\n👤 Converting employment length:")
        
        # Check original data
        print(f"   Original unique values: {df_clean['emp_length'].value_counts().index.tolist()}")
        
        # Create mapping
        emp_length_mapping = {
            '< 1 year': 0.5,
            '1 year': 1,
            '2 years': 2,
            '3 years': 3,
            '4 years': 4,
            '5 years': 5,
            '6 years': 6,
            '7 years': 7,
            '8 years': 8,
            '9 years': 9,
            '10+ years': 10,
            'n/a': np.nan
        }
        
        # Convert
        df_clean['emp_length_numeric'] = df_clean['emp_length'].map(emp_length_mapping)
        
        print(f"   Converted unique values: {df_clean['emp_length_numeric'].value_counts().index.tolist()}")
        print(f"   Data type: {df_clean['emp_length_numeric'].dtype}")
        conversions_made += 1
    
    # 4. Convert loan term to numeric
    if 'term' in df_clean.columns:
        print(f"\n💰 Converting loan term:")
        
        # Check original data
        print(f"   Original unique values: {df_clean['term'].value_counts().index.tolist()}")
        
        # Extract numeric value
        df_clean['term_months'] = df_clean['term'].str.extract('(\d+)').astype(float)
        
        print(f"   Converted unique values: {df_clean['term_months'].value_counts().index.tolist()}")
        print(f"   Data type: {df_clean['term_months'].dtype}")
        conversions_made += 1
    
    # 5. Handle zip code (convert to first 3 digits for region analysis)
    if 'zip_code' in df_clean.columns:
        print(f"\n📍 Processing zip code:")
        
        # Extract first 3 digits
        df_clean['zip_code_3'] = df_clean['zip_code'].str[:3]
        
        print(f"   Converted zip codes to first 3 digits for region analysis")
        print(f"   Unique regions: {df_clean['zip_code_3'].nunique()}")
        conversions_made += 1
    
    print(f"\n✅ Data type conversion completed!")
    print(f"   Total conversions made: {conversions_made}")
    print(f"   Dataset shape after conversion: {df_clean.shape}")
    
    # Update main dataframe
    df = df_clean
    
else:
    print("❌ Dataset not available")

## 4. Handle Missing Values

In [ ]:
if df is not None:
    print("="*80)
    print("STEP 3: HANDLE MISSING VALUES")
    print("="*80)
    
    # Analyze missing values
    missing_counts = df.isnull().sum()
    missing_percentages = (missing_counts / len(df) * 100)
    
    # Create missing values DataFrame
    missing_df = pd.DataFrame({
        'Column': df.columns,
        'Missing Count': missing_counts.values,
        'Missing %': missing_percentages.values,
        'Data Type': df.dtypes.values
    })
    
    missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing %', ascending=False)
    
    print(f"\n📊 Missing Values Analysis:")
    print(f"   Columns with missing values: {len(missing_df)}")
    print(f"   Total missing values: {missing_counts.sum():,}")
    
    # Strategy 1: Drop columns with >50% missing
    print(f"\n🗑️  Strategy 1: Drop columns with >50% missing values")
    high_missing_cols = missing_df[missing_df['Missing %'] > 50]['Column'].tolist()
    
    if high_missing_cols:
        print(f"   Dropping {len(high_missing_cols)} columns:")
        for col in high_missing_cols:
            missing_pct = missing_df[missing_df['Column'] == col]['Missing %'].iloc[0]
            print(f"     - {col}: {missing_pct:.1f}% missing")
        
        df.drop(columns=high_missing_cols, inplace=True)
        print(f"   ✅ Dropped {len(high_missing_cols)} columns")
    else:
        print("   ✅ No columns with >50% missing values")
    
    # Strategy 2: Handle important columns with missing values
    print(f"\n🔧 Strategy 2: Handle important columns with missing values")
    
    # Update missing_df after dropping columns
    missing_counts = df.isnull().sum()
    missing_percentages = (missing_counts / len(df) * 100)
    missing_df = pd.DataFrame({
        'Column': df.columns,
        'Missing Count': missing_counts.values,
        'Missing %': missing_percentages.values,
        'Data Type': df.dtypes.values
    })
    missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing %', ascending=False)
    
    # Define important columns and their handling strategy
    important_cols_strategies = {
        'emp_title': {'strategy': 'fill_unknown', 'fill_value': 'Unknown'},
        'emp_length_numeric': {'strategy': 'fill_median'},
        'annual_inc': {'strategy': 'fill_median'},
        'dti': {'strategy': 'fill_median'},
        'delinq_2yrs': {'strategy': 'fill_zero'},
        'inq_last_6mths': {'strategy': 'fill_zero'},
        'open_acc': {'strategy': 'fill_median'},
        'pub_rec': {'strategy': 'fill_zero'},
        'revol_bal': {'strategy': 'fill_median'},
        'revol_util': {'strategy': 'fill_median'},
        'total_acc': {'strategy': 'fill_median'},
        'collections_12_mths_ex_med': {'strategy': 'fill_zero'},
        'acc_now_delinq': {'strategy': 'fill_zero'},
        'tot_coll_amt': {'strategy': 'fill_zero'},
        'tot_cur_bal': {'strategy': 'fill_median'},
        'total_rev_hi_lim': {'strategy': 'fill_median'},
    }
    
    # Apply strategies
    handled_cols = 0
    
    for col, strategy_info in important_cols_strategies.items():
        if col in df.columns and df[col].isnull().sum() > 0:
            strategy = strategy_info['strategy']
            missing_count = df[col].isnull().sum()
            missing_pct = missing_count / len(df) * 100
            
            print(f"   📝 Processing {col}: {missing_count:,} ({missing_pct:.1f}%) missing")
            
            if strategy == 'fill_unknown':
                df[col] = df[col].fillna(strategy_info['fill_value'])
                print(f"     Filled with '{strategy_info['fill_value']}'")
            
            elif strategy == 'fill_median':
                median_val = df[col].median()
                df[col] = df[col].fillna(median_val)
                print(f"     Filled with median: {median_val}")
            
            elif strategy == 'fill_zero':
                df[col] = df[col].fillna(0)
                print(f"     Filled with 0")
            
            elif strategy == 'fill_mode':
                mode_val = df[col].mode().iloc[0] if not df[col].mode().empty else 'Unknown'
                df[col] = df[col].fillna(mode_val)
                print(f"     Filled with mode: {mode_val}")
            
            handled_cols += 1
    
    # Strategy 3: Handle remaining categorical columns
    print(f"\n📝 Strategy 3: Handle remaining categorical columns")
    
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    remaining_categorical_missing = [col for col in categorical_cols if df[col].isnull().sum() > 0]
    
    for col in remaining_categorical_missing:
        missing_count = df[col].isnull().sum()
        missing_pct = missing_count / len(df) * 100
        print(f"   📝 Processing {col}: {missing_count:,} ({missing_pct:.1f}%) missing")
        
        # Fill with mode or 'Unknown'
        if df[col].nunique() < 20:  # Low cardinality
            fill_value = df[col].mode().iloc[0] if not df[col].mode().empty else 'Unknown'
        else:  # High cardinality
            fill_value = 'Unknown'
        
        df[col] = df[col].fillna(fill_value)
        print(f"     Filled with: {fill_value}")
        handled_cols += 1
    
    # Strategy 4: Handle remaining numerical columns
    print(f"\n📊 Strategy 4: Handle remaining numerical columns")
    
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    remaining_numerical_missing = [col for col in numerical_cols if df[col].isnull().sum() > 0]
    
    for col in remaining_numerical_missing:
        missing_count = df[col].isnull().sum()
        missing_pct = missing_count / len(df) * 100
        print(f"   📊 Processing {col}: {missing_count:,} ({missing_pct:.1f}%) missing")
        
        # Fill with median for numerical columns
        fill_value = df[col].median()
        df[col] = df[col].fillna(fill_value)
        print(f"     Filled with median: {fill_value}")
        handled_cols += 1
    
    # Final check
    final_missing = df.isnull().sum().sum()
    print(f"\n✅ Missing values handling completed!")
    print(f"   Columns handled: {handled_cols}")
    print(f"   Remaining missing values: {final_missing:,}")
    print(f"   Dataset shape: {df.shape}")
    
    if final_missing == 0:
        print(f"   🎉 No missing values remaining!")
    else:
        print(f"   ⚠️  Still has {final_missing} missing values")
        remaining_missing_cols = df.columns[df.isnull().any()].tolist()
        print(f"   Remaining columns with missing: {remaining_missing_cols}")
    
else:
    print("❌ Dataset not available")

## 5. Feature Engineering

In [ ]:
if df is not None:
    print("="*80)
    print("STEP 4: FEATURE ENGINEERING")
    print("="*80)
    
    engineered_features = 0
    
    print("\n🔧 Creating engineered features...")
    
    # 1. FICO Score Features
    fico_cols = [col for col in df.columns if 'fico' in col.lower()]
    if 'fico_range_low' in df.columns and 'fico_range_high' in df.columns:
        print(f"\n📈 Creating FICO score features:")
        
        # FICO Average
        df['fico_avg'] = (df['fico_range_low'] + df['fico_range_high']) / 2
        print(f"   ✅ fico_avg: Average FICO score")
        print(f"      Range: {df['fico_avg'].min():.0f} - {df['fico_avg'].max():.0f}")
        engineered_features += 1
        
        # FICO Score Categories
        df['fico_category'] = pd.cut(
            df['fico_avg'],
            bins=[0, 680, 720, 760, 850],
            labels=['Fair (680-)', 'Good (680-720)', 'Very Good (720-760)', 'Excellent (760+)']
        )
        print(f"   ✅ fico_category: FICO score categories")
        print(f"      Categories: {df['fico_category'].value_counts().index.tolist()}")
        engineered_features += 1
        
        # Low FICO Flag
        df['low_fico_flag'] = (df['fico_avg'] < 680).astype(int)
        print(f"   ✅ low_fico_flag: FICO < 680")
        print(f"      Low FICO loans: {df['low_fico_flag'].sum():,} ({df['low_fico_flag'].mean()*100:.1f}%)")
        engineered_features += 1
    
    # 2. Financial Ratio Features
    if all(col in df.columns for col in ['loan_amnt', 'annual_inc']):
        print(f"\n💰 Creating financial ratio features:")
        
        # Loan to Income Ratio
        df['loan_to_income_ratio'] = df['loan_amnt'] / (df['annual_inc'] / 12)
        print(f"   ✅ loan_to_income_ratio: Loan amount / monthly income")
        print(f"      Range: {df['loan_to_income_ratio'].min():.1f} - {df['loan_to_income_ratio'].max():.1f}")
        engineered_features += 1
        
        # Monthly Debt Burden
        if 'installment' in df.columns:
            df['monthly_debt_burden'] = df['installment'] / (df['annual_inc'] / 12)
            print(f"   ✅ monthly_debt_burden: Monthly payment / monthly income")
            print(f"      Range: {df['monthly_debt_burden'].min():.3f} - {df['monthly_debt_burden'].max():.3f}")
            engineered_features += 1
    
    # 3. Enhanced DTI Features
    if all(col in df.columns for col in ['dti', 'installment', 'annual_inc']):
        print(f"\n📊 Creating enhanced DTI features:")
        
        # Effective DTI (including new loan)
        monthly_income = df['annual_inc'] / 12
        new_payment_dti = (df['installment'] / monthly_income) * 100
        df['effective_dti'] = df['dti'] + new_payment_dti
        print(f"   ✅ effective_dti: Current DTI + new loan payment ratio")
        print(f"      Range: {df['effective_dti'].min():.1f} - {df['effective_dti'].max():.1f}")
        engineered_features += 1
        
        # High DTI Flags
        df['high_dti_flag'] = (df['dti'] > 30).astype(int)
        df['very_high_dti_flag'] = (df['dti'] > 40).astype(int)
        print(f"   ✅ high_dti_flag: DTI > 30% ({df['high_dti_flag'].sum():,} loans)")
        print(f"   ✅ very_high_dti_flag: DTI > 40% ({df['very_high_dti_flag'].sum():,} loans)")
        engineered_features += 2
    
    # 4. Credit History Features
    if 'earliest_cr_line' in df.columns and 'issue_d' in df.columns:
        print(f"\n📅 Creating credit history features:")
        
        # Credit Age in Years
        df['credit_age_years'] = (df['issue_d'] - df['earliest_cr_line']).dt.days / 365.25
        df['credit_age_years'] = df['credit_age_years'].fillna(0)  # Handle missing dates
        print(f"   ✅ credit_age_years: Length of credit history in years")
        print(f"      Range: {df['credit_age_years'].min():.1f} - {df['credit_age_years'].max():.1f}")
        engineered_features += 1
        
        # Credit Age Categories
        df['credit_age_category'] = pd.cut(
            df['credit_age_years'],
            bins=[0, 5, 10, 20, 100],
            labels=['<5 years', '5-10 years', '10-20 years', '20+ years']
        )
        print(f"   ✅ credit_age_category: Credit history length categories")
        engineered_features += 1
    
    # 5. Employment Stability Features
    if 'emp_length_numeric' in df.columns:
        print(f"\n👤 Creating employment stability features:")
        
        # Employment Stability Score
        if 'home_ownership' in df.columns:
            df['employment_stability_score'] = (
                df['emp_length_numeric'] * 7 +  # Employment length weight
                (df['home_ownership'].isin(['MORTGAGE', 'OWN']) * 20) +  # Home ownership bonus
                (df.get('verification_status', '').isin(['Source Verified', 'Verified']) * 13)  # Verification bonus
            )
            print(f"   ✅ employment_stability_score: Combined employment stability metric")
            print(f"      Range: {df['employment_stability_score'].min():.0f} - {df['employment_stability_score'].max():.0f}")
            engineered_features += 1
        
        # Stable Employment Flag
        df['stable_employment_flag'] = (df['emp_length_numeric'] >= 5).astype(int)
        print(f"   ✅ stable_employment_flag: 5+ years employment ({df['stable_employment_flag'].sum():,} loans)")
        engineered_features += 1
    
    # 6. Credit Utilization Features
    if 'revol_util' in df.columns:
        print(f"\n💳 Creating credit utilization features:")
        
        # High Utilization Flags
        df['high_utilization_flag'] = (df['revol_util'] > 80).astype(int)
        df['very_high_utilization_flag'] = (df['revol_util'] > 90).astype(int)
        print(f"   ✅ high_utilization_flag: Revolving utilization > 80% ({df['high_utilization_flag'].sum():,} loans)")
        print(f"   ✅ very_high_utilization_flag: Revolving utilization > 90% ({df['very_high_utilization_flag'].sum():,} loans)")
        engineered_features += 2
        
        # Utilization Categories
        df['utilization_category'] = pd.cut(
            df['revol_util'],
            bins=[0, 30, 60, 80, 100],
            labels=['Low (0-30%)', 'Medium (30-60%)', 'High (60-80%)', 'Very High (80-100%)']
        )
        print(f"   ✅ utilization_category: Credit utilization categories")
        engineered_features += 1
    
    # 7. Delinquency Risk Features
    delinquency_cols = [col for col in df.columns if 'delinq' in col.lower() or 'pub_rec' in col.lower()]
    if 'delinq_2yrs' in df.columns:
        print(f"\n⚠️  Creating delinquency risk features:")
        
        # Recent Delinquency Flag
        df['recent_delinquency_flag'] = (df['delinq_2yrs'] > 0).astype(int)
        print(f"   ✅ recent_delinquency_flag: Any delinquency in last 2 years ({df['recent_delinquency_flag'].sum():,} loans)")
        engineered_features += 1
        
        # Multiple Delinquencies Flag
        df['multiple_delinquencies_flag'] = (df['delinq_2yrs'] > 1).astype(int)
        print(f"   ✅ multiple_delinquencies_flag: 2+ delinquencies in last 2 years ({df['multiple_delinquencies_flag'].sum():,} loans)")
        engineered_features += 1
    
    # 8. Inquiry Risk Features
    if 'inq_last_6mths' in df.columns:
        print(f"\n🔍 Creating inquiry risk features:")
        
        # Recent Inquiries Flag
        df['recent_inquiries_flag'] = (df['inq_last_6mths'] > 0).astype(int)
        print(f"   ✅ recent_inquiries_flag: Any inquiries in last 6 months ({df['recent_inquiries_flag'].sum():,} loans)")
        engineered_features += 1
        
        # High Inquiries Flag
        df['high_inquiries_flag'] = (df['inq_last_6mths'] > 3).astype(int)
        print(f"   ✅ high_inquiries_flag: 4+ inquiries in last 6 months ({df['high_inquiries_flag'].sum():,} loans)")
        engineered_features += 1
    
    print(f"\n✅ Feature engineering completed!")
    print(f"   Total engineered features created: {engineered_features}")
    print(f"   New dataset shape: {df.shape}")
    print(f"   Total columns: {len(df.columns)}")
    
    # Show new features summary
    if engineered_features > 0:
        print(f"\n📋 New Features Summary:")
        # Get all newly created columns (not in original dataset)
        original_cols = set(df_original.columns)
        current_cols = set(df.columns)
        new_cols = current_cols - original_cols
        
        print(f"   Total new features: {len(new_cols)}")
        for i, col in enumerate(sorted(list(new_cols)), 1):
            if df[col].dtype in ['object', 'category']:
                unique_vals = df[col].nunique()
                print(f"   {i:2d}. {col:<30} (categorical, {unique_vals} unique values)")
            else:
                min_val = df[col].min()
                max_val = df[col].max()
                print(f"   {i:2d}. {col:<30} (numeric, {min_val:.2f} to {max_val:.2f})")
    
else:
    print("❌ Dataset not available")

## 6. Handle Outliers

In [ ]:
if df is not None:
    print("="*80)
    print("STEP 5: HANDLE OUTLIERS")
    print("="*80)
    
    # Define outlier handling strategies
    outlier_strategies = {
        'annual_inc': {'method': 'cap', 'upper_percentile': 99.5, 'min_value': 20000, 'max_value': 500000},
        'loan_amnt': {'method': 'cap', 'min_value': 1000, 'max_value': 35000},
        'dti': {'method': 'cap', 'min_value': 0, 'max_value': 50},
        'revol_util': {'method': 'cap', 'min_value': 0, 'max_value': 120},  # Allow some >100% for overlimit cases
        'int_rate': {'method': 'cap', 'min_value': 0, 'max_value': 30},
        'loan_to_income_ratio': {'method': 'cap', 'max_value': 10},  # Max 10x monthly income
        'monthly_debt_burden': {'method': 'cap', 'max_value': 0.5},  # Max 50% of monthly income
        'effective_dti': {'method': 'cap', 'max_value': 60},  # Max 60% including new loan
    }
    
    outliers_handled = 0
    
    print("\n🔍 Analyzing and handling outliers...")
    
    for col, strategy in outlier_strategies.items():
        if col in df.columns:
            method = strategy['method']
            
            print(f"\n📊 Processing {col}:")
            
            # Show original statistics
            original_min = df[col].min()
            original_max = df[col].max()
            original_mean = df[col].mean()
            original_median = df[col].median()
            
            print(f"   Original range: {original_min:,.2f} to {original_max:,.2f}")
            print(f"   Original mean: {original_mean:,.2f}, median: {original_median:,.2f}")
            
            if method == 'cap':
                # Calculate bounds
                if 'upper_percentile' in strategy:
                    upper_bound = df[col].quantile(strategy['upper_percentile'] / 100)
                elif 'max_value' in strategy:
                    upper_bound = strategy['max_value']
                else:
                    upper_bound = df[col].quantile(0.99)
                
                if 'lower_percentile' in strategy:
                    lower_bound = df[col].quantile(strategy['lower_percentile'] / 100)
                elif 'min_value' in strategy:
                    lower_bound = strategy['min_value']
                else:
                    lower_bound = df[col].quantile(0.01)
                
                # Count outliers
                upper_outliers = (df[col] > upper_bound).sum()
                lower_outliers = (df[col] < lower_bound).sum()
                total_outliers = upper_outliers + lower_outliers
                
                print(f"   Outlier bounds: {lower_bound:,.2f} to {upper_bound:,.2f}")
                print(f"   Outliers detected: {total_outliers:,} ({total_outliers/len(df)*100:.2f}%)")
                print(f"     Lower outliers: {lower_outliers:,}")
                print(f"     Upper outliers: {upper_outliers:,}")
                
                # Apply capping
                df[col] = df[col].clip(lower_bound, upper_bound)
                
                # Show new statistics
                new_min = df[col].min()
                new_max = df[col].max()
                new_mean = df[col].mean()
                new_median = df[col].median()
                
                print(f"   New range: {new_min:,.2f} to {new_max:,.2f}")
                print(f"   New mean: {new_mean:,.2f}, median: {new_median:,.2f}")
                print(f"   ✅ Outliers handled with capping method")
                
                outliers_handled += total_outliers
    
    # Additional outlier detection using IQR method for important features
    print(f"\n📈 Additional outlier check using IQR method:")
    
    important_features = ['annual_inc', 'loan_amnt', 'dti', 'int_rate', 'fico_avg']
    iqr_outliers = 0
    
    for col in important_features:
        if col in df.columns:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 3 * IQR  # More conservative (3*IQR instead of 1.5)
            upper_bound = Q3 + 3 * IQR
            
            outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
            outlier_pct = outliers / len(df) * 100
            
            if outlier_pct > 0.1:  # Only show if significant
                print(f"   {col}: {outliers:,} outliers ({outlier_pct:.2f}%)")
                iqr_outliers += outliers
    
    if iqr_outliers == 0:
        print("   No significant outliers detected with IQR method")
    
    print(f"\n✅ Outlier handling completed!")
    print(f"   Total outliers processed: {outliers_handled:,}")
    print(f"   Dataset shape: {df.shape}")
    
    # Visualize before/after for key variables
    key_vars = ['annual_inc', 'loan_amnt', 'dti']
    key_vars = [var for var in key_vars if var in df.columns]
    
    if key_vars:
        fig, axes = plt.subplots(len(key_vars), 2, figsize=(15, 5*len(key_vars)))
        if len(key_vars) == 1:
            axes = [axes]
        
        for i, var in enumerate(key_vars):
            # Compare with original if available
            if var in df_original.columns:
                # Original data
                axes[i][0].hist(df_original[var].dropna(), bins=50, alpha=0.7, color='red', label='Original')
                axes[i][0].set_title(f'{var} - Original Distribution')
                axes[i][0].set_xlabel(var)
                axes[i][0].set_ylabel('Frequency')
                axes[i][0].legend()
                
                # Cleaned data
                axes[i][1].hist(df[var].dropna(), bins=50, alpha=0.7, color='green', label='Cleaned')
                axes[i][1].set_title(f'{var} - Cleaned Distribution')
                axes[i][1].set_xlabel(var)
                axes[i][1].set_ylabel('Frequency')
                axes[i][1].legend()
            else:
                # Just show cleaned data
                axes[i][0].hist(df[var].dropna(), bins=50, alpha=0.7, color='blue')
                axes[i][0].set_title(f'{var} - Distribution')
                axes[i][0].set_xlabel(var)
                axes[i][0].set_ylabel('Frequency')
                
                # Box plot
                axes[i][1].boxplot(df[var].dropna())
                axes[i][1].set_title(f'{var} - Box Plot')
                axes[i][1].set_ylabel(var)
        
        plt.tight_layout()
        plt.show()
    
else:
    print("❌ Dataset not available")

## 7. Final Quality Check and Summary

In [ ]:
if df is not None:
    print("="*80)
    print("DATA CLEANING - FINAL SUMMARY")
    print("="*80)
    
    # Basic statistics
    print(f"\n📊 FINAL DATASET SUMMARY:")
    print(f"   Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"   Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    # Target variable check
    if 'loan_status_binary' in df.columns:
        target_counts = df['loan_status_binary'].value_counts()
        default_rate = target_counts[1] / len(df) * 100
        print(f"   Target variable (loan_status_binary):")
        print(f"     Fully Paid (0): {target_counts[0]:,} ({target_counts[0]/len(df)*100:.1f}%)")
        print(f"     Charged Off (1): {target_counts[1]:,} ({default_rate:.1f}%)")
        print(f"     Default rate: {default_rate:.2f}%")
    
    # Missing values check
    total_missing = df.isnull().sum().sum()
    cols_with_missing = df.isnull().any().sum()
    print(f"   Missing values: {total_missing:,} total in {cols_with_missing} columns")
    
    # Data types summary
    dtype_counts = df.dtypes.value_counts()
    print(f"   Data types:")
    for dtype, count in dtype_counts.items():
        print(f"     {dtype}: {count} columns")
    
    # Key engineered features
    if 'fico_avg' in df.columns:
        print(f"   FICO scores: {df['fico_avg'].min():.0f} - {df['fico_avg'].max():.0f}")
    if 'loan_to_income_ratio' in df.columns:
        print(f"   Loan-to-income ratios: {df['loan_to_income_ratio'].min():.1f} - {df['loan_to_income_ratio'].max():.1f}")
    if 'effective_dti' in df.columns:
        print(f"   Effective DTI: {df['effective_dti'].min():.1f}% - {df['effective_dti'].max():.1f}%")
    
    # Quality metrics
    print(f"\n✅ DATA QUALITY METRICS:")
    print(f"   Completeness: {((1 - total_missing/(df.shape[0]*df.shape[1]))*100):.1f}%")
    print(f"   Duplicate rows: {df.duplicated().sum():,}")
    
    # Check for any remaining data issues
    issues_found = []
    
    # Check for infinite values
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    infinite_count = np.isinf(df[numeric_cols]).sum().sum()
    if infinite_count > 0:
        issues_found.append(f"Infinite values: {infinite_count:,}")
    
    # Check for negative values where they shouldn't exist
    non_negative_cols = ['loan_amnt', 'annual_inc', 'int_rate', 'dti', 'revol_bal', 'fico_avg']
    for col in non_negative_cols:
        if col in df.columns:
            negative_count = (df[col] < 0).sum()
            if negative_count > 0:
                issues_found.append(f"Negative {col}: {negative_count:,}")
    
    if issues_found:
        print(f"   ⚠️  Issues found: {len(issues_found)}")
        for issue in issues_found:
            print(f"     - {issue}")
    else:
        print(f"   🎉 No data quality issues detected!")
    
    # Save cleaned dataset
    print(f"\n💾 Saving cleaned dataset...")
    
    try:
        # Create data/processed directory if it doesn't exist
        os.makedirs('../data/processed', exist_ok=True)
        
        # Save cleaned data
        cleaned_file = '../data/processed/loan_data_cleaned.csv'
        df.to_csv(cleaned_file, index=False)
        print(f"   ✅ Cleaned dataset saved to: {cleaned_file}")
        print(f"   File size: {os.path.getsize(cleaned_file) / 1024**2:.1f} MB")
        
        # Save cleaning log
        cleaning_log = {
            'cleaning_date': datetime.now().isoformat(),
            'original_shape': df_original.shape,
            'cleaned_shape': df.shape,
            'records_removed': df_original.shape[0] - df.shape[0],
            'columns_added': df.shape[1] - df_original.shape[1],
            'target_variable': 'loan_status_binary',
            'default_rate': default_rate if 'loan_status_binary' in df.columns else None,
            'missing_values_handled': total_missing,
            'outliers_handled': outliers_handled if 'outliers_handled' in locals() else 0,
            'engineered_features': engineered_features if 'engineered_features' in locals() else 0
        }
        
        with open('../data/processed/cleaning_log.json', 'w') as f:
            json.dump(cleaning_log, f, indent=2, default=str)
        print(f"   ✅ Cleaning log saved to: ../data/processed/cleaning_log.json")
        
    except Exception as e:
        print(f"   ❌ Error saving cleaned dataset: {e}")
    
    print(f"\n🎉 DATA CLEANING COMPLETED SUCCESSFULLY!")
    print(f"="*50)
    print(f"✅ Dataset is now ready for feature engineering and modeling")
    print(f"📝 Next step: Proceed to '03_feature_engineering.ipynb'")
    print(f"📊 Or use the cleaned dataset directly for modeling")
    
    # Show sample of cleaned data
    print(f"\n📋 Sample of Cleaned Data (First 3 rows, Selected Columns):")
    display_cols = ['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'fico_avg', 
                   'loan_status_binary', 'loan_to_income_ratio', 'effective_dti']
    display_cols = [col for col in display_cols if col in df.columns]
    
    if display_cols:
        display(df[display_cols].head(3).round(2))
    
else:
    print("❌ Dataset not available for final summary")